In [2]:
import pandas as pd
import os 
import json
import csv
from tqdm import tqdm
from pydub import AudioSegment
import numpy as np
import librosa
import soundfile as sf
import random

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>train_count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Sylvia atricapilla_Eurasian Blackcap</th>
      <td>2702</td>
    </tr>
    <tr>
      <th>Fringilla coelebs_Common Chaffinch</th>
      <td>2632</td>
    </tr>
    <tr>
      <th>Turdus philomelos_Song Thrush</th>
      <td>2360</td>
    </tr>
    <tr>
      <th>Troglodytes troglodytes_Eurasian Wren</th>
      <td>1828</td>
    </tr>
    <tr>
      <th>Periparus ater_Coal Tit</th>
      <td>1399</td>
    </tr>
    <tr>
      <th>Erithacus rubecula_European Robin</th>
      <td>1259</td>
    </tr>
    <tr>
      <th>Regulus ignicapilla_Common Firecrest</th>
      <td>1237</td>
    </tr>
    <tr>
      <th>Turdus merula_Eurasian Blackbird</th>
      <td>1162</td>
    </tr>
    <tr>
      <th>Phylloscopus collybita_Common Chiffchaff</th>
      <td>1042</td>
    </tr>
    <tr>
      <th>Phylloscopus trochilus_Willow Warbler</th>
      <td>670</td>
    </tr>
    <tr>
      <th>Regulus regulus_Goldcrest</th>
      <td>600</td>
    </tr>
    <tr>
      <th>Turdus viscivorus_Mistle Thrush</th>
      <td>304</td>
    </tr>
    <tr>
      <th>Rain_</th>
      <td>223</td>
    </tr>
    <tr>
      <th>Wind_</th>
      <td>158</td>
    </tr>
    <tr>
      <th>Dendrocopos major_Great Spotted Woodpecker</th>
      <td>148</td>
    </tr>
    <tr>
      <th>Lophophanes cristatus_Crested Tit</th>
      <td>139</td>
    </tr>
    <tr>
      <th>Muscicapa striata_Spotted Flycatcher</th>
      <td>136</td>
    </tr>
    <tr>
      <th>Loxia curvirostra_Common Crossbill</th>
      <td>128</td>
    </tr>
    <tr>
      <th>Pecking_</th>
      <td>87</td>
    </tr>
    <tr>
      <th>Certhia familiaris_Eurasian Treecreeper</th>
      <td>80</td>
    </tr>
    <tr>
      <th>Dryocopus martius_Black Woodpecker</th>
      <td>68</td>
    </tr>
    <tr>
      <th>Vegetation_</th>
      <td>46</td>
    </tr>
  </tbody>
</table>
</div>

## move 20 samples for each specie in validation

In [ ]:
# move 20 samples for each specie in validation
valid_folder = "E:/Giacomo/Tovanella/DATASET/validation"
train_folder = "E:/Giacomo/Tovanella/orig_segments/train"
for specie in os.listdir(train_folder):
    length_train_audio = len(os.listdir(os.path.join(train_folder, specie)))
    threshold = int(length_train_audio / 100 * 12)
    audio_samples = np.random.choice(os.listdir(os.path.join(train_folder, specie)), threshold, replace=False)
    # for audio in audio_samples:
    #     os.rename(
    #         os.path.join(train_folder, specie, audio),
    #         os.path.join(valid_folder, specie, audio)
    #     )

In [157]:
for specie in os.listdir(valid_folder):
    for audio in os.listdir(os.path.join(valid_folder, specie)):
        os.rename(
            os.path.join(valid_folder, specie, audio),
            os.path.join(train_folder, specie, audio)
        )

In [10]:
train_folder = "E:/Giacomo/Tovanella/orig_segments/train"
valid_folder = "E:/Giacomo/Tovanella/DATASET/validation"
test_folder = "E:/Giacomo/Tovanella/DATASET/test"
species_folders = os.listdir(train_folder)
species_count = { folder: {
    "train_count": len(os.listdir(os.path.join(train_folder, folder))),
    "valid_count": len(os.listdir(os.path.join(valid_folder, folder))),
    "test_count": len(os.listdir(os.path.join(test_folder, folder))),
    } for folder in species_folders }
df = pd.DataFrame(species_count).T
df.sort_values(by="test_count", ascending=True)

,train_count,valid_count,test_count
Pecking_,77,10,4
Rain_,198,27,6
Certhia familiaris_Eurasian Treecreeper,91,11,7
Dendrocopos major_Great Spotted Woodpecker,134,16,7
Lophophanes cristatus_Crested Tit,126,15,8
Dryocopus martius_Black Woodpecker,62,8,9
Vegetation_,45,6,17
Erithacus rubecula_European Robin,1100,133,18
Loxia curvirostra_Common Crossbill,118,16,18
Troglodytes troglodytes_Eurasian Wren,1595,198,20


In [4]:
# validation back to training folder
for specie in os.listdir(test_folder):
    for audio in os.listdir(os.path.join(test_folder, specie)):
        if len(audio.split("_")) > 3:
            os.rename(
                os.path.join(test_folder, specie, audio),
                os.path.join(train_folder, specie, audio)
            )

### move sample into Other folder until 200 sample in training folder (random)

In [15]:
## move sample into Other folder until 200 sample in training folder (random)

train_folder = "E:/Giacomo/Tovanella/orig_segments/train"
removed_folder = "E:/Giacomo/Tovanella/orig_segments/removed"

species_folders = os.listdir(train_folder)

for specie in species_folders:
    if len(os.listdir(os.path.join(train_folder, specie))) <= 600:
        continue
    audio_samples = np.random.choice(os.listdir(os.path.join(train_folder, specie)), 600, replace=False)
    os.makedirs(os.path.join(removed_folder, specie), exist_ok=True)
    for audio in os.listdir(os.path.join(train_folder, specie)):
        if audio in audio_samples:    
            continue
        os.rename(
            os.path.join(train_folder, specie, audio),
            os.path.join(removed_folder, specie, audio)
        )

In [ ]:
## ripristina tutto in training folder

train_folder = "E:/Giacomo/Tovanella/orig_segments/train"
removed_folder = "E:/Giacomo/Tovanella/orig_segments/removed"

species_folders = os.listdir(train_folder)

for specie in species_folders:
    if not os.path.exists(os.path.join(removed_folder, specie)):
        continue
    # for audio in os.listdir(os.path.join(removed_folder, specie)):
    #     if audio in audio_samples:    
    #         continue
    #     os.rename(
    #         os.path.join(removed_folder, specie, audio),
    #         os.path.join(train_folder, specie, audio)
    #     )

In [11]:
# print proportion train and valid set
train_folder = "E:/Giacomo/Tovanella/orig_segments/train"
test_folder = "E:/Giacomo/Tovanella/DATASET/test"
valid_folder = "E:/Giacomo/Tovanella/DATASET/validation"

train_count = {}
valid_count = {}
test_count = {}
for specie in os.listdir(train_folder):
    train_count[specie] = len(os.listdir(os.path.join(train_folder, specie))) 
    valid_count[specie] = len(os.listdir(os.path.join(valid_folder, specie))) 
    test_count[specie] = len(os.listdir(os.path.join(test_folder, specie))) 

split_info = {}
for specie in train_count.keys():
    len_sum = train_count[specie] + valid_count[specie] + test_count[specie]
    split_info[specie] = {
        "train": train_count[specie] / len_sum,
        "valid": valid_count[specie] / len_sum,
        "test": test_count[specie] / len_sum,
        "total": len_sum
    }
info = pd.DataFrame(split_info).T
info

,train,valid,test,total
Certhia familiaris_Eurasian Treecreeper,0.834862,0.100917,0.064220,109.0
Dendrocopos major_Great Spotted Woodpecker,0.853503,0.101911,0.044586,157.0
Dryocopus martius_Black Woodpecker,0.784810,0.101266,0.113924,79.0
Erithacus rubecula_European Robin,0.876099,0.106315,0.017586,1251.0
Fringilla coelebs_Common Chaffinch,0.632039,0.086119,0.281843,3321.0
Lophophanes cristatus_Crested Tit,0.845638,0.100671,0.053691,149.0
Loxia curvirostra_Common Crossbill,0.776316,0.105263,0.118421,152.0
Muscicapa striata_Spotted Flycatcher,0.058394,0.109489,0.832117,137.0
Pecking_,0.846154,0.109890,0.043956,91.0
Periparus ater_Coal Tit,0.868293,0.108711,0.022997,1435.0


In [12]:
true_segments_test = {}
for specie in os.listdir(test_folder):
    for audio in os.listdir(os.path.join(test_folder, specie)):
        # CLH_20200215_090000_18.WAV
        if len(audio.split("_")) == 4:
            site, date, number, segm_wav = audio.split("_")
            audio_name = "_".join([site, date, number]) + ".WAV"
        else:
            date, number, segm_wav = audio.split("_")
            audio_name = "_".join([date, number]) + ".WAV"
        segm = segm_wav.split(".")[0]
        if audio_name not in true_segments_test:
            true_segments_test[audio_name] = {}
        if segm not in true_segments_test[audio_name]:
            true_segments_test[audio_name][segm] = []
        true_segments_test[audio_name][segm].append(specie)

for audio in true_segments_test:
    for segm in true_segments_test[audio]:
        true_segments_test[audio][segm] = list(set(true_segments_test[audio][segm]))
true_segments_test

{'20190608_070000.WAV': {'88': ['Certhia familiaris_Eurasian Treecreeper'],
  '101': ['Regulus ignicapilla_Common Firecrest'],
  '105': ['Regulus ignicapilla_Common Firecrest'],
  '129': ['Regulus ignicapilla_Common Firecrest'],
  '50': ['Regulus ignicapilla_Common Firecrest'],
  '72': ['Regulus ignicapilla_Common Firecrest'],
  '74': ['Regulus ignicapilla_Common Firecrest']},
 '20190608_090000.WAV': {'103': ['Certhia familiaris_Eurasian Treecreeper'],
  '99': ['Certhia familiaris_Eurasian Treecreeper']},
 '20200215_110000.WAV': {'129': ['Certhia familiaris_Eurasian Treecreeper'],
  '164': ['Lophophanes cristatus_Crested Tit'],
  '165': ['Lophophanes cristatus_Crested Tit'],
  '166': ['Lophophanes cristatus_Crested Tit'],
  '167': ['Lophophanes cristatus_Crested Tit'],
  '68': ['Wind_'],
  '69': ['Wind_'],
  '70': ['Wind_']},
 '20200217_160000.WAV': {'196': ['Certhia familiaris_Eurasian Treecreeper'],
  '197': ['Certhia familiaris_Eurasian Treecreeper'],
  '198': ['Certhia familiaris_E

In [16]:
true_segments_valid = {}
for specie in os.listdir(valid_folder):
    for audio in os.listdir(os.path.join(valid_folder, specie)):
        # CLH_20200215_090000_18.WAV
        if len(audio.split("_")) == 4:
            site, date, number, segm_wav = audio.split("_")
            audio_name = "_".join([site, date, number]) + ".WAV"
        else:
            date, number, segm_wav = audio.split("_")
            audio_name = "_".join([date, number]) + ".WAV"
        segm = segm_wav.split(".")[0]
        if audio_name not in true_segments_valid:
            true_segments_valid[audio_name] = {}
        if segm not in true_segments_valid[audio_name]:
            true_segments_valid[audio_name][segm] = []
        true_segments_valid[audio_name][segm].append(specie)

for audio in true_segments_valid:
    for segm in true_segments_valid[audio]:
        true_segments_valid[audio][segm] = list(set(true_segments_valid[audio][segm]))
true_segments_valid

{'20190603_030000.WAV': {'190': ['Certhia familiaris_Eurasian Treecreeper'],
  '193': ['Certhia familiaris_Eurasian Treecreeper'],
  '3': ['Periparus ater_Coal Tit'],
  '36': ['Periparus ater_Coal Tit'],
  '66': ['Periparus ater_Coal Tit'],
  '86': ['Regulus regulus_Goldcrest', 'Periparus ater_Coal Tit'],
  '10': ['Regulus ignicapilla_Common Firecrest'],
  '110': ['Regulus ignicapilla_Common Firecrest'],
  '111': ['Regulus ignicapilla_Common Firecrest'],
  '139': ['Regulus ignicapilla_Common Firecrest'],
  '146': ['Regulus ignicapilla_Common Firecrest'],
  '16': ['Regulus ignicapilla_Common Firecrest'],
  '195': ['Regulus ignicapilla_Common Firecrest'],
  '54': ['Regulus ignicapilla_Common Firecrest'],
  '6': ['Regulus ignicapilla_Common Firecrest', 'Regulus regulus_Goldcrest'],
  '61': ['Regulus ignicapilla_Common Firecrest'],
  '65': ['Regulus ignicapilla_Common Firecrest'],
  '84': ['Regulus ignicapilla_Common Firecrest'],
  '93': ['Regulus ignicapilla_Common Firecrest', 'Regulus re

In [14]:
with open("utils/true_segments_test.json", "w") as f:
    json.dump(true_segments_test, f)

In [15]:
with open("utils/true_segments_valid.json", "w") as f:
    json.dump(true_segments_valid, f)